### csv.reader

In [236]:
import csv
from subprocess import call
import pandas as pd
import numpy as np

In [51]:
!pwd

/home/jlewis425/Seattle_g89/anomalies-in-earnings-release-reactions/data


In [73]:
filename = 'stock_vol_data.csv'

In [240]:
def create_vol_features(raw_file):
    """docstring"""
    
    # open combined_clean.csv
    df = pd.read_csv('combined_clean.csv', low_memory=False)
    tickers = list(df['ticker_symbol'])
    dates = list(df['t-7'])
    codes = list(df['unique_earnings_code'])
    
    
    # create containers for vol data
    vol_spread1 = []
    vol_spread2 = []
    hist_v_imp = []
    
    
    # open raw data file
    with open(raw_file, "r") as raw:
        datareader = csv.reader(raw)
        
        for row in datareader:
            ticker = row[0]
            date = row[1]
            
            if ticker in tickers and date in dates:
                
                # first vol spread calc
                try:
                    vs1 = float(row[3]) - float(row[7])
                
                except ValueError:
                    vs1 = np.nan
                
                vol_spread1.append(vs1)
                
                # second vol spread calc
                try:
                    vs2 = float(row[2]) - float(row[5])
                    
                except ValueError:
                    vs2 = np.nan
                
                vol_spread2.append(vs2)
                
                # historical vs implied calc
                try:
                    hvi = float(row[4]) - float(row[27])
                except ValueError:
                    hvi = np.nan
                    
                hist_v_imp.append(hvi)
                
    new_frame_cols = [codes, tickers, dates, vol_spread1, vol_spread2, hist_v_imp]
    vol_frame = pd.DataFrame(new_frame_cols).T
    
    return vol_frame

In [241]:
path = '/home/jlewis425/Seattle_g89/anomalies-in-earnings-release-reactions/vol_data/'
filename = 'VOL_20190401.csv'
target = path+filename

In [242]:
vol_frame = create_vol_features(target)

In [246]:
vol_frame.head()

,0,1,2,3,4,5
0,TCO.1Q14,TCO,2014-04-18,-0.0826,-0.1463,-0.0364
1,SNX.1Q14,SNX,2014-03-28,-0.081,-0.2138,-0.0429
2,AIV.1Q14,AIV,2014-04-25,-0.0793,-0.2089,-0.0421
3,IT.1Q14,IT,2014-04-24,-0.0691,-0.1123,-0.03
4,MPWR.1Q14,MPWR,2014-04-18,-0.0899,-0.1331,-0.0287


In [269]:
vol_frame_clean = vol_frame.loc[~vol_frame.iloc[:, 0].isnull(), :]

In [270]:
vol_frame_clean.head()

,0,1,2,3,4,5
0,TCO.1Q14,TCO,2014-04-18,-0.0826,-0.1463,-0.0364
1,SNX.1Q14,SNX,2014-03-28,-0.081,-0.2138,-0.0429
2,AIV.1Q14,AIV,2014-04-25,-0.0793,-0.2089,-0.0421
3,IT.1Q14,IT,2014-04-24,-0.0691,-0.1123,-0.03
4,MPWR.1Q14,MPWR,2014-04-18,-0.0899,-0.1331,-0.0287


In [271]:
column_names = ['unique_earnings_code', 'ticker_symbol', 't-7', 'hvol_20_120_F', 'hvol_10_60_F', 'h30_v_imp30_F']

In [272]:
vol_frame_clean.set_axis(column_names, axis=1, inplace=True)

In [273]:
vol_frame_clean.set_index('unique_earnings_code', inplace=True)

In [275]:
vol_frame_clean.drop(columns=['ticker_symbol', 't-7'], inplace=True)

/home/jlewis425/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [277]:
vol_frame_clean.to_csv('clean_vol_data')

### Dask 

In [10]:
import dask.dataframe as dd
from pandas import read_json

In [25]:
vol_col_names = read_json('vol_col_names.json', typ='series')

In [26]:
col_names = list(vol_col_names.index)

In [31]:
col_names.insert(0,'ticker_symbol')

In [34]:
col_names.insert(1, 'date')

In [41]:
df.head(4)

,ticker_symbol,date,Hv10,Hv20,Hv30,Hv60,Hv90,Hv120,Hv150,Hv180,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
0,A,2013-01-03,0.2613,0.2588,0.2549,0.2732,0.2634,0.2509,0.275,0.2783,...,0.2926,0.0286,0.3003,0.3252,0.3128,0.021,0.3013,0.3269,0.3141,0.0206
1,A,2013-01-04,0.276,0.2497,0.2599,0.2743,0.2659,0.2527,0.2713,0.2745,...,0.2897,0.0289,0.2924,0.3261,0.3092,0.0196,0.2929,0.328,0.3104,0.0191
2,A,2013-01-07,0.2812,0.2551,0.2618,0.273,0.2664,0.2525,0.2715,0.2744,...,0.2908,0.0278,0.3001,0.3242,0.3121,0.021,0.3009,0.3257,0.3133,0.0206
3,A,2013-01-08,0.2886,0.2475,0.2613,0.2691,0.2663,0.2511,0.2691,0.2746,...,0.2895,0.0289,0.3019,0.3168,0.3093,0.0286,0.3028,0.3179,0.3103,0.0287


### pymongo

In [110]:
import csv
import json
import pandas as pd
import sys, getopt, pprint

import pymongo
from pymongo import MongoClient

In [132]:
def _create_headers():
    """HELPER to create field names for pymongo collection"""
    # read file
    with open('vol_col_names.json', 'r') as myfile:
        data=myfile.read()

    # parse file
    vol_labels = json.loads(data)
    
    headers = vol_labels.keys()
    headers = list(headers)
    headers.insert(0, "ticker_symbol")
    headers.insert(1, 'date')
    
    return headers

In [142]:
def csv_to_mongo(csv_filename, db_name):
    
    # instantiate client
    client = MongoClient()
    
    # connect to (or create) mongo db
    db = client[db_name]
    # create collection
    stocks = db.stocks        # is this right?
    
    # create headers
    header = _create_headers()
    
    #CSV to JSON Conversion
    csvfile = open(csv_filename)
    reader = csv.DictReader(csvfile, fieldnames=header)
    
    
    for each in reader:
        row={}
        for field in header:
            
            row[field]=each[field] # needs proper formatting?
            
        db.stocks.insert_one(row)


In [143]:
csv_to_mongo('sample_stock_vol_data.csv', 'test')